In [1]:
from requests import get
import bs4
from bs4 import BeautifulSoup


import pandas as pd
import numpy as np

from time import sleep
from random import randint

from time import time
from warnings import warn

from datetime import datetime
from datetime import timedelta
from pytz import timezone

from IPython.display import clear_output


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

import re
import os
import sys

import glob
import shutil
from pathlib import Path

In [66]:
# Path of the data folder
folder_data = r'C:\Users\motodangiuuuu\Crawler'

# Settings for web driver and Proxy
chrome_driver_path = os.path.join(os.getcwd(),'chromedriver.exe')
# PROXY = "socks5://localhost:9050" # IP:PORT or HOST:PORT

# create a new Chrome session
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
# options.add_argument('--headless')
# options.add_argument('--proxy-server=%s' % PROXY)
driver = webdriver.Chrome(executable_path=chrome_driver_path, options=options)
driver.implicitly_wait(0)


In [67]:
url = 'https://careerbuilder.vn/vi/tim-viec-lam/sales-executive-b2b.35B5EE5D.html?fbclid=IwAR3scscHshSUX8rTIJIXiHwy__ij3u20AZpyf9DXetFCUkBnvoeuAy8DCAk'
driver.get(url)
sleep(3)

In [68]:
dict_detail = {}

In [75]:
#Hàm lấy thông tin tổng quan của công ty
def get_company_info(link_company_info, dict_detail):
    driver.get(link_company_info)
    sleep(5)
    
    element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="company-info"]')))
    html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
    soup_level1=BeautifulSoup(html_of_interest, 'lxml')  

    company_info = soup_level1.select('div[class="content"]')[0]

    name = company_info.select('p')[0].text
    address = company_info.select('p')[1].text
    dict_detail['Tên công ty'] = name
    dict_detail['Địa chỉ'] = address

    company_detail = company_info.select('ul > li')
    for i in range(len(company_detail)):
        role = company_detail[i].text.split(":")[0].strip()
        detail = company_detail[i].text.split(":")[1].strip()
        dict_detail[role] = detail
    return dict_detail

In [76]:
#Hàm lấy thông tin Phúc lợi của công ty
def get_welfares_info(soup_level1, dict_detail):
    welfares = soup_level1.select('div[class="detail-row box-welfares"]')[0].select('li')
    phuc_loi=[]
    for i in welfares:
        phuc_loi.append(i.text.strip())
        dict_detail['Phúc lợi'] = phuc_loi
    return dict_detail

In [87]:
#Hàm lấy thông tin Phúc lợi của công ty => chỉ dùng block 3, 4, 5
def get_job_tags(soup_level1, dict_detail):
    job_tags = soup_level1.select('div[class="job-tags detail-row"]')[0]
    jobtags_skills=[]
    tag_skills = job_tags.select('li')
    if tag_skills:
        for tag in tag_skills:
            jobtags_skills.append(tag.text)
    dict_detail['Job tags / Skills'] = jobtags_skills  
    return dict_detail

In [72]:
element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section[class="template template04 "]')))
html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
soup_level1=BeautifulSoup(html_of_interest, 'lxml')

In [73]:
map_div = soup_level1.select('div[class="head-left"]')[0].select('div[class="title"] > h2')
dict_detail['Tên việc làm'] = map_div[0].text
dict_detail['Tên việc làm']

'Sales Executive (B2B)'

In [78]:
dict_detail['Link'] = url

In [77]:
#Thông tin tuyển dụng
map_div = soup_level1.select('div[class="box-info"]')[0].select('td')
i=0
while i < len(map_div):
    dict_detail[map_div[i].text.strip()] = map_div[i+1].text.strip()
    i += 2

In [79]:
get_welfares_info(soup_level1, dict_detail)
# welfares = soup_level1.select('div[class="detail-row box-welfares"]')[0].select('li')
# phuc_loi=[]
# for i in welfares:
#     phuc_loi.append(i.text.strip())
#     dict_detail['Phúc lợi'] = phuc_loi
# dict_detail['Phúc lợi']

{'Tên việc làm': 'Sales Executive (B2B)',
 'Link': 'https://careerbuilder.vn/vi/tim-viec-lam/sales-executive-b2b.35B5EE5D.html?fbclid=IwAR3scscHshSUX8rTIJIXiHwy__ij3u20AZpyf9DXetFCUkBnvoeuAy8DCAk',
 'Ngành nghề': 'Bán hàng / Kinh doanh,                                    Dịch vụ khách hàng',
 'Lương': 'Cạnh tranh',
 'Hình thức': 'Nhân viên chính thức',
 'Cấp bậc': 'Nhân viên',
 'Kinh nghiệm': '2 - 3\n                                                Năm',
 'Hết hạn nộp': '30/04/2021',
 'Địa điểm': 'Hồ Chí Minh',
 'Phúc lợi': ['Chế độ bảo hiểm', 'Chế độ thưởng', 'Đào tạo', 'Tăng lương']}

In [80]:
map_div = soup_level1.select('div[class="full-content"]')[0].select('div[class="detail-row"]')
map_div[0]

<div class="detail-row">
<div class="content">
<p><strong>1. </strong><strong>Develop business and keep retention clients</strong></p><ul><li>Maintain and develop sales volume;</li><li>Develop new business opportunities from new market segments, existing clients and renewals;</li><li>Understand features, advantages, disadvantages of the products &amp; services as well as competitors’ one;</li><li>Contact with customers and record all customers’ information in the report of customer’s information;</li><li>Keep close relationship for retention of clients;</li><li>Co-operate with sales admin to prepare contract, service executive form, check list…to get customers;</li><li>Planning for week work, month for approval and comply with the approved plans;</li><li>Receive and handle complaints from customers regarding to product quality, delivery time, etc;</li><li>Inspect the premises thoroughly for signs of pests, identify the pest and find out where they are entering the premises.</li><li>Coo

In [81]:
#Mô tả công việc
att =""
for item in map_div[0].select('p, ul'):
    att = att + str(item) +("\n")
    att = att.replace("<", "[").replace(">", "]")
dict_detail["Mô tả Công việc"] = att

In [82]:
for item in range(1, len(map_div)):
    attribute = map_div[item]  
    att =""
    for item in attribute.select('p, ul'):
        att = att + str(item) +("\n")
        att = att.replace("<", "[").replace(">", "]")
    dict_detail[attribute.select('h3')[0].text.strip().capitalize()] = att

In [43]:
# # element = driver.find_element_by_tag_name("td")
# # element.click()
# path_cralwer = r'C:\Users\motodangiuuuu\Crawler\Data'
# csvvv = pd.read_csv(os.path.join(path_cralwer, "0_to_60_jobs.csv"))
# csvvv[:10]

In [84]:
link_company_info = soup_level1.select('a[class="company"]')[0]['href']
# link_company_info
get_company_info(link_company_info, dict_detail)

{'Tên việc làm': 'Sales Executive (B2B)',
 'Link': 'https://careerbuilder.vn/vi/tim-viec-lam/sales-executive-b2b.35B5EE5D.html?fbclid=IwAR3scscHshSUX8rTIJIXiHwy__ij3u20AZpyf9DXetFCUkBnvoeuAy8DCAk',
 'Ngành nghề': 'Bán hàng / Kinh doanh,                                    Dịch vụ khách hàng',
 'Lương': 'Cạnh tranh',
 'Hình thức': 'Nhân viên chính thức',
 'Cấp bậc': 'Nhân viên',
 'Kinh nghiệm': '2 - 3\n                                                Năm',
 'Hết hạn nộp': '30/04/2021',
 'Địa điểm': 'Hồ Chí Minh',
 'Phúc lợi': ['Chế độ bảo hiểm', 'Chế độ thưởng', 'Đào tạo', 'Tăng lương'],
 'Mô tả Công việc': '[p][strong]1. [/strong][strong]Develop business and keep retention clients[/strong][/p]\n[ul][li]Maintain and develop sales volume;[/li][li]Develop new business opportunities from new market segments, existing clients and renewals;[/li][li]Understand features, advantages, disadvantages of the products &amp; services as well as competitors’ one;[/li][li]Contact with customers and recor

In [59]:
element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="company-info"]')))
html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
soup_level1=BeautifulSoup(html_of_interest, 'lxml')  

In [60]:
company_info = soup_level1.select('div[class="content"]')[0]

In [61]:
name = company_info.select('p')[0].text
address = company_info.select('p')[1].text
dict_detail['Tên công ty'] = name
dict_detail['Địa chỉ'] = address

In [62]:
company_detail = company_info.select('ul > li')
company_detail
for i in range(len(company_detail)):
    role = company_detail[i].text.split(":")[0].strip()
    detail = company_detail[i].text.split(":")[1].strip()
    dict_detail[role] = detail

In [85]:
df = pd.DataFrame.from_dict(dict_detail, orient='index')
df = df.transpose()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Tên việc làm         1 non-null      object
 1   Link                 1 non-null      object
 2   Ngành nghề           1 non-null      object
 3   Lương                1 non-null      object
 4   Hình thức            1 non-null      object
 5   Cấp bậc              1 non-null      object
 6   Kinh nghiệm          1 non-null      object
 7   Hết hạn nộp          1 non-null      object
 8   Địa điểm             1 non-null      object
 9   Phúc lợi             1 non-null      object
 10  Mô tả Công việc      1 non-null      object
 11  Yêu cầu công việc    1 non-null      object
 12  Thông tin khác       1 non-null      object
 13  Job tags / Skills    1 non-null      object
 14  Tên công ty          1 non-null      object
 15  Địa chỉ              1 non-null      object
 16  Qui mô công 

In [86]:
df

Tên việc làm                                               Link  \
0  Sales Executive (B2B)  https://careerbuilder.vn/vi/tim-viec-lam/sales...   

                                          Ngành nghề       Lương  \
0  Bán hàng / Kinh doanh,                        ...  Cạnh tranh   

              Hình thức    Cấp bậc  \
0  Nhân viên chính thức  Nhân viên   

                                         Kinh nghiệm Hết hạn nộp     Địa điểm  \
0  2 - 3\n                                       ...  30/04/2021  Hồ Chí Minh   

                                            Phúc lợi  \
0  [Chế độ bảo hiểm, Chế độ thưởng, Đào tạo, Tăng...   

                                     Mô tả Công việc  \
0  [p][strong]1. [/strong][strong]Develop busines...   

                                   Yêu cầu công việc  \
0  [ul][li]Bachelor of engineering degree or in e...   

                                      Thông tin khác  \
0  [ul]\n[li] Bằng cấp:\n                Đại học\...   

                                   Job tags / Skills  \
0  [ Sales Supervisor,  Nhân viên kinh doanh,  Ch...   

                                Tên công ty  \
0  Công ty TNHH Rentokil Initial (Việt Nam)   

                                             Địa chỉ Qui mô công ty  \
0  Tầng 4 tòa nhà Aloha - Số 68 đường Hồng Hà, Ph...            295   

   Loại hình hoạt động                   Website  
0  Công ty đa quốc gia  www.rentokil-initial.com